# Preliminary Imports

In [3]:
import pandas as pd
import numpy as np
from random import randrange
from functools import reduce
from tail_recursive import tail_recursive

# Part 1: Think About The Data (Chris)

## Part 1.1

Our dataset offers childcare prices from the year 2008 to 2018 by childcare provider type, age of children, and county characteristics. This data set is of particular interest to us because it provides meaningful insight into how childcare costs have changed over the years and how it was affected by changing governing bodies in the United States. Based on such interpretation, the data set can be an effective tool for present and future parents and policymakers when making a decision that will impact childcare prices.

## Part 1.2

There are a total of 62 numerical attributes and 3 categorical attributes in the data set.

## Part 1.3

There are missing values in our data set, often for specific age groups or entire counties. 

(Complete tomorrow during the meeting (9/26))


## Part 1.4

We believe that the “unemployment rate of the population aged 20 to 64 years old”, “median household income expressed in 2018 dollars” and “county identifier”will be the most descriptive of the data. This is because we believe that the subgroup of relatively high median household income and low unemployment rate will inevitably be able to afford high-quality childcare, thus leading to purchasing higher-priced childcare. Similarly, subgroups of relatively low median household income and high unemployment rate will lead to lower-priced childcare. For county identifier, we believe that as different states have different policies regarding childcare, this will inevitably lead to subgroups of states / counties with similar policies.

## Part 1.5

We believe that there will be clusters in the data because some of the attributes, like those mentioned in the previous section, are heavy influences of childcare prices, thus we expect there to be clusters of similar groups of those attributes (i.e., unemployment rate).

## Part 1.6

By finding clusters in the data, we can gain an understanding of the influential attributes that have led to the clusters. This is massively help parents and policymakers to understand how their decisions might affect childcare prices. Furthermore, with new instances, we can have a better understanding of which cluster it will fall into. 

## Part 1.7

(NO IDEA)

## Part 1.8

This will depend on what type of clustering algorithm we implement. However, in general, we do not expect that the clusters will be of similar size. This is because our dataset contains a large number of numerical attributes. Furthermore, taking the example of county identifier as an example, the number of states / counties that share similar childcare policies will not be evenly distributed, thus leading to clusters of different sizes.

In [9]:
df = pd.read_csv("childcare_costs.csv")
#df.index.to_series().map(lambda i: df.iloc[i].isna().sum() / len(df.iloc[i])).to_csv("nancount.csv")

In [10]:
df.describe()

,county_fips_code,study_year,unr_16,funr_16,munr_16,unr_20to64,funr_20to64,munr_20to64,flfpr_20to64,flfpr_20to64_under6,...,memp_p,femp_p,mcsa,mfccsa,mc_infant,mc_toddler,mc_preschool,mfcc_infant,mfcc_toddler,mfcc_preschool
count,34567.000000,34567.000000,34567.000000,34567.00000,34567.000000,34567.000000,34567.000000,34567.000000,34567.000000,34567.000000,...,34567.000000,34567.000000,23593.000000,23383.000000,23593.000000,23593.000000,23593.000000,23383.000000,23383.000000,23383.000000
mean,30388.132786,2012.999711,7.465902,7.02902,7.860291,6.900073,6.482007,7.275457,70.086125,68.821409,...,23.456238,7.634403,101.234253,92.523582,146.051770,130.482768,122.232852,113.421657,106.759749,104.189510
std,15161.015383,3.162232,3.538619,3.56342,4.037657,3.446199,3.477956,3.990758,7.696499,11.758088,...,7.956927,4.501044,34.552888,27.669904,53.698566,43.775370,38.538323,32.819372,29.982431,28.961701
min,1001.000000,2008.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,33.600000,0.000000,...,0.000000,0.000000,18.980000,22.000000,27.730000,21.540000,21.540000,43.080000,43.080000,40.030000
25%,18177.000000,2010.000000,5.100000,4.64000,5.200000,4.600000,4.200000,4.700000,65.100000,62.600000,...,17.610000,4.410000,78.650000,75.000000,108.750000,100.000000,95.880000,90.000000,85.085000,84.255000
50%,29177.000000,2013.000000,7.050000,6.59000,7.390000,6.500000,6.000000,6.800000,70.600000,69.600000,...,23.400000,6.630000,96.530000,88.180000,134.500000,120.990000,113.990000,106.000000,100.250000,99.650000
75%,45081.000000,2016.000000,9.350000,8.88000,9.920000,8.700000,8.250000,9.200000,75.500000,76.100000,...,28.930000,9.920000,119.380000,107.500000,166.330000,148.710000,139.300000,129.315000,124.950000,120.200000
max,56045.000000,2018.000000,36.110000,38.24000,39.740000,33.900000,44.500000,45.500000,100.000000,100.000000,...,87.500000,66.480000,375.400000,308.000000,470.000000,419.000000,385.000000,430.940000,376.320000,331.340000


# Part 2: Perform Some Explanatory Data Analysis

## Part 2.1 (Qasim)

## Part 2.2 (Fatima)

## Part 2.3 (Fatima)

## Part 2.4 (Qasim)

## Part 2.5 (Qasim)

## Part 2.6 (Qasim)

## Part 2.7 (Qasim)

## Part 2.8 (Fatima)

## Part 2.9 (Qasim)

# Part 3: Write Functions For Clustering in Python

## Part 3.1: K-Means Algorithm (Ian)

### Preliminaries

I am using a small data set to test on. This data corresponds to soybean data.

In [7]:
df = pd.read_csv("soybean_data.csv", index_col=0)

### Create Distance Matrix Function

In [2]:
def distMatrix(df, centroids):
    df_mat, cent_mat = df.to_numpy(), centroids.to_numpy()
    return np.array([[np.linalg.norm(df_mat[i, :] - cent_mat[j, :]) for j in range(cent_mat.shape[0])] for i in range(df_mat.shape[0])])

### Assign New Clusters

In [4]:
def cluster(dm):
    def f(i):
        return reduce(lambda x, y: x if x[0] <= y[0] else y, zip(dm[i, :], range(len(dm[i, :]))))[1]
    return f

### Bringing It All Together

In [5]:
def kmeans(df, k, eps):
    @tail_recursive
    def go(currentClus, prevClus, r):
        print("Iteration Number {}".format(r))
        centroids = pd.DataFrame(df.to_dict(orient='series') | {"Cluster": currentClus}).groupby(by=["Cluster"]).mean()
        if prevClus is not None and ((currentClus != prevClus).sum() / len(currentClus)) < eps:
            return centroids
        else:
            dm = distMatrix(df, centroids)
            return go.tail_call(df.index.map(cluster(dm)), currentClus, r + 1)
    return go([randrange(k) for i in range(df.shape[0])], None, 0)

In [8]:
kmeans(df, 4, 0.001)

Iteration Number 0
Iteration Number 1
Iteration Number 2
Iteration Number 3
Iteration Number 4
Iteration Number 5


,Date,Plant-Stand,Precip,Temp,Hail,Crop-Hist,Area-Damaged,Severity,Seed-TMT,Germination,...,Int-Discolor,Sclerotia,Fruit-Pods,Fruit Spots,Seed,Mold-Growth,Seed-Discolor,Seed-Size,Shriveling,Roots
Cluster,,,,,,,,,,,,,,,,,,,,,
0,4.700000,0.000000,0.000000,1.600000,0.600000,1.600000,2.5,1.000000,0.500000,0.900000,...,2.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.800000,1.000000,1.800000,0.600000,0.300000,1.100000,1.2,1.500000,0.500000,0.300000,...,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,1.000000
2,4.500000,0.000000,2.000000,1.000000,0.100000,1.900000,0.3,1.300000,0.500000,1.300000,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,1.588235,0.882353,1.882353,0.235294,0.294118,2.058824,1.0,1.647059,0.470588,1.647059,...,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.470588


## Part 3.2

# Part 4: Analyze Your Data

## Part 4.1

## Part 4.2 (Qasim)

## Part 4.3

## Part 4.4